# Tabular Playground Series - Oct 2021

#### Oct 01, 2021 to Oct 31, 2021

#### https://www.kaggle.com/c/tabular-playground-series-oct-2021/

#### _**Predicting the biological response of molecules given various chemical properties**_

Notebook Author:

| Name  | Pradip Kumar Das  |
| ------------: | :------------ |
| **Profile:**  | [LinkedIn](https://www.linkedin.com/in/daspradipkumar/ "LinkedIn") l [GitHub](https://github.com/PradipKumarDas "GitHub") l [Kaggle](https://www.kaggle.com/pradipkumardas "Kaggle")  |
| **Contact:**  | pradipkumardas@hotmail.com (Email)  |
| **Location:**  | Bengaluru, India  |

**Sections:**

* Dependencies
* Exploratory Data Analysis (EDA) & Preprocessing
* Modeling & Evaluation
* Submission

## Dependencies

In [1]:
# Loads required packages

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier
import xgboost as xgb

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

## Exploratory Data Analysis (EDA) & Preprocessing

In [2]:
# Loads train dataset
train = pd.read_csv("../input/tabular-playground-series-oct-2021/train.csv")

In [3]:
# Checks how the train data set looks
with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
    display(train)

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
0,0,0.205979,0.410993,0.176775,0.223581,0.423543,0.476140,0.413590,0.612021,0.534873,0.147295,0.026177,0.106613,0.200924,0.713191,0.155750,0.557335,0.341702,0.285720,0.230396,0.203957,0.509588,0.706972,1,0.007793,0.247765,0.263750,0.259555,0.231730,0.138379,0.197824,0.054392,0.194153,0.281500,0.034818,0.025334,0.114432,0.139203,0.246157,0.251371,0.701423,0.301182,0.193924,0.267497,0,0.193430,0.238630,0.154770,0.249857,0.210685,0.406662,0.214810,0.258668,0.377518,0.192042,0.340855,0.199660,0.264074,0.205550,0.075109,0.205688,0.178962,0.245008,0.519336,0.306419,0.127139,0.367479,0.236380,0.195694,0.013195,0.199588,0.283367,0.168824,0.004855,0.117723,0.257688,0.197262,0.211452,0.372637,0.198157,0.689970,0.449955,0.713110,0.212041,0.183619,0.288667,0.648678,0.600398,0.223267,0.590163,0.248847,0.795641,0.139932,0.618696,0.639142,0.008549,0.559151,0.573640,0.138808,0.499156,0.112203,0.181498,0.165887,0.093171,0.106952,0.127861,0.250924,0.501673,0.036740,0.111361,0.075918,0.019444,0.250760,0.465093,0.087502,0.004185,0.195936,0.166389,0.171328,0.146014,0.199232,0.133999,0.168191,0.010242,0.294490,0.012977,0.003969,0.013739,0.040076,0.170711,0.250246,0.195538,0.708556,0.448925,0.550352,0.217984,0.751629,0.822459,0.186298,0.024197,0.044097,0.078943,0.181147,0.022591,0.576712,0.406843,0.510578,0.799434,0.651125,0.460708,0.636714,0.350704,0.872989,0.007751,0.017103,0.019875,0.203042,0.864594,0.595877,0.542969,0.990250,0.020373,0.006238,0.011040,0.006019,0.407014,0.080140,0.013502,0.144265,0.007229,0.003256,0.014556,0.123806,0.133871,0.011531,0.010250,0.253750,0.090162,0.147857,0.303087,0.112764,0.104344,0.168583,0.011342,0.239028,0.008018,0.167653,0.217342,0.184178,0.179060,0.078009,0.135768,0.006534,0.009832,0.013317,0.390079,0.004801,0.056600,0.114139,0.012599,0.014818,0.446073,0.216079,0.152113,0.111237,0.170896,0.190477,0.011936,0.005227,0.429740,0.013060,0.199369,0.258410,0.208863,0.129545,0.006978,0.012948,0.049466,0.008804,0.114205,0.119683,0.191210,0.169976,0.188199,0.355674,0.013164,0.304878,0.007213,0.011277,0.083186,0.010624,0.031199,0.200306,0.195791,0.203470,0.036314,0.157711,0.199117,0.007444,0.189048,0.202540,0.273267,0.167211,1,0,1,1,1,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,1,1,0,0,0,1,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1
1,1,0.181004,0.473119,0.011734,0.213657,0.619678,0.441593,0.230407,0.686013,0.281971,0.238509,0.493411,0.107277,0.231828,0.457150,0.395200,0.617088,0.459358,0.209225,0.201098,0.199383,0.366578,0.585788,1,0.285311,0.400367,0.162493,0.249365,0.141160,0.133688,0.247906,0.139251,0.216444,0.109674,0.033018,0.017458,0.189336,0.168785,0.184251,0.202753,0.218451,0.324364,0.255499,0.287433,0,0.821982,0.284351,0.177537,0.207

In [7]:
# Drops ID column as it is not required
train.drop(["id"], axis=1, inplace=True)

In [4]:
# Checks for data types used in the data set
train.dtypes.unique()

array([dtype('int64'), dtype('float64')], dtype=object)

In [5]:
# Checks for nubmer of row having any missing values ('0' indicates no rows have missing values)
sum(train.isna().sum())

0

In [6]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                     df[col] = df[col].astype(np.float16)
#                 el
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [8]:
# Compresses the training data as Kaggle kernel resets due to large size of the training data 
train = reduce_mem_usage(train)

Memory usage of dataframe is 2182.01 MB
Memory usage after optimization is: 959.40 MB
Decreased by 56.0%


In [9]:
# Shows the column data types after data compression
train.dtypes

f0        float32
f1        float32
f2        float32
f3        float32
f4        float32
           ...   
f281         int8
f282         int8
f283         int8
f284         int8
target       int8
Length: 286, dtype: object

In [10]:
# Checks distribution of categorical target variable
train.target.value_counts()

1    500485
0    499515
Name: target, dtype: int64

**As `target` is equaly distributed, it itself can be used as bins in stratified K-Fold validation**

## Modeling & Evaluation

In [11]:
# Seperates predictor variables from target

y = train.target
train.drop(["target"], axis=1, inplace=True)

In [12]:
# Create stratification object for K-Fold cross validation
sk_fold = StratifiedKFold(n_splits=5)

In [13]:
# Performs cross validation on XGB Classifier

cv_generator = sk_fold.split(train, y)

model = XGBClassifier(
    n_estimators=100,
    objective='binary:logistic', 
    eval_metric='auc',
    tree_method='gpu_hist'
)

cv_scores = cross_val_score(model, train, y, scoring='roc_auc', cv=cv_generator, n_jobs=-1, verbose=10)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:  1.0min remaining:   41.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.3min finished


In [14]:
print("ROC AUC score of XGBoost (with default parameters) Model:", cv_scores.mean())

ROC AUC score of XGBoost (with default parameters) Model: 0.8510695329013235


In [15]:
del cv_scores, model, cv_generator

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV]  ................................................................
[CV] .................................... , score=0.851, total=  33.3s
[CV]  ................................................................
[CV] .................................... , score=0.849, total=  26.4s
[CV]  ................................................................
[CV] .................................... , score=0.853, total=  33.3s
[CV]  ................................................................
[CV] .................................... , score=0.853, total=  26.2s
[CV]  ................................................................
[CV] .................................... , score=0.849, total=  13.1s


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encode

**Automated Hyperparameter Tuning with Hyperopt**

In [22]:
# Instead of performing cross validation during hyperparameter tunining, 
# the tuning is done over fixed train and validation data set to save significant amount of time
# The following code snippet extract that stratified set of train and validation set

cv_generator = sk_fold.split(train, y)

for fold, (idx_train, idx_val) in enumerate(cv_generator):
    y_val = y.iloc[idx_val]
    dtrain = xgb.DMatrix(data=train.iloc[idx_train], label=y.iloc[idx_train])
    dval = xgb.DMatrix(data=train.iloc[idx_val], label=y.iloc[idx_val])
    break

In [18]:
# Sets up a search space for XGBoost hyperparameters
space = {
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'max_depth': hp.quniform("max_depth", 2, 6, 1),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 8, 1),
    'reg_alpha' : hp.uniform('reg_alpha', 1e-8, 100),
    'reg_lambda' : hp.uniform('reg_lambda', 1e-8, 100),
    'gamma': hp.uniform ('gamma', 0.0, 1.0),
    'subsample': hp.uniform("subsample", 0.1, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1.0)
}

In [28]:
def trial_loss(space):
    """
    Trial function for Hyperopt to call by passing a set a trial hyperparamets
    to train model and perform predictions.
    
    Parameters:
    ----------
    space: A set a trial hyperparamets
    
    Returns metric for Hyperopt to estimate for further tuning in search space.
    """
    
    # Converts parameter value to int as required by XGBoost
    space["max_depth"] = int(space["max_depth"])
    space["objective"] = "binary:logistic"
    space["eval_metric"] = "auc"
    space["tree_method"] = "gpu_hist"
    
    model = xgb.train(
        space, 
        dtrain, 
        num_boost_round=2000, 
        evals=[(dtrain, 'train'), (dval, 'eval')],
        early_stopping_rounds=50, verbose_eval=False)
    
    predictions = model.predict(dval)
    
    roc_auc = roc_auc_score(y_val, predictions)
    
    del predictions, model, space
    
    return {"loss": -roc_auc, "status": STATUS_OK}

In [29]:
# Starts hyperparameters tuning
trials = Trials()
best_trial = fmin(fn=trial_loss, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

100%|██████████| 50/50 [45:43<00:00, 54.87s/trial, best loss: -0.8563927977799835]


In [30]:
# Views the best hyperparameters
best_trial

{'colsample_bytree': 0.5509440040918419,
 'gamma': 0.2749290513553893,
 'learning_rate': 0.055610269598680366,
 'max_depth': 4.0,
 'min_child_weight': 8.0,
 'reg_alpha': 97.43879743549955,
 'reg_lambda': 75.11623552980602,
 'subsample': 0.844084987947726}

In [31]:
del dtrain, dval, y_val, cv_generator

## Submission

In [32]:
# Loads test data set
test = pd.read_csv("../input/tabular-playground-series-oct-2021/test.csv")

# Removes ID column as it is not required for prediction
test.drop(["id"], axis=1, inplace=True)

# Loads submission data set that acts just as a template for submission
submission = pd.read_csv("../input/tabular-playground-series-oct-2021/sample_submission.csv")

**Prepares final XGBoost model with optimized parameters**

In [34]:
# Adds other important parameters
best_trial["max_depth"] = int(best_trial["max_depth"])
best_trial["objective"] = "binary:logistic"
best_trial["eval_metric"] = "auc"
best_trial["tree_method"] = "gpu_hist"

In [43]:
# Gets the model trained over cross validation and predictions 
# against each iteration is stored

test_predictions = []

cv_generator = sk_fold.split(train, y)

dtest = xgb.DMatrix(data=test)

for fold, (idx_train, idx_val) in enumerate(cv_generator):
    print("fold", fold)

    dtrain = xgb.DMatrix(data=train.iloc[idx_train], label=y.iloc[idx_train])
    dval = xgb.DMatrix(data=train.iloc[idx_val], label=y.iloc[idx_val])
    
    model = xgb.train(
        best_trial, 
        dtrain, 
        num_boost_round=2000, 
        evals=[(dtrain, 'train'), (dval, 'eval')],
        early_stopping_rounds=50, verbose_eval=200)
    
    predictions = model.predict(dtest)
    
    test_predictions.append(predictions)
    
    del predictions, model, dval, dtrain

fold 0
[0]	train-auc:0.59823	eval-auc:0.59433
[200]	train-auc:0.84836	eval-auc:0.84682
[400]	train-auc:0.85485	eval-auc:0.85184
[600]	train-auc:0.85810	eval-auc:0.85391
[800]	train-auc:0.86015	eval-auc:0.85494
[1000]	train-auc:0.86164	eval-auc:0.85555
[1200]	train-auc:0.86279	eval-auc:0.85591
[1400]	train-auc:0.86377	eval-auc:0.85612
[1600]	train-auc:0.86462	eval-auc:0.85628
[1800]	train-auc:0.86539	eval-auc:0.85634
[1999]	train-auc:0.86610	eval-auc:0.85639
fold 1
[0]	train-auc:0.59664	eval-auc:0.59901
[200]	train-auc:0.84758	eval-auc:0.84919
[400]	train-auc:0.85414	eval-auc:0.85405
[600]	train-auc:0.85741	eval-auc:0.85602
[800]	train-auc:0.85949	eval-auc:0.85688
[1000]	train-auc:0.86100	eval-auc:0.85732
[1200]	train-auc:0.86217	eval-auc:0.85753
[1400]	train-auc:0.86315	eval-auc:0.85759
[1437]	train-auc:0.86331	eval-auc:0.85761
fold 2
[0]	train-auc:0.59790	eval-auc:0.59533
[200]	train-auc:0.84776	eval-auc:0.84795
[400]	train-auc:0.85424	eval-auc:0.85328
[600]	train-auc:0.85740	eval-auc

In [54]:
test_predictions

[array([0.7308241 , 0.25364307, 0.9059377 , ..., 0.2811955 , 0.5688376 ,
        0.39456698], dtype=float32),
 array([0.75234586, 0.26519367, 0.88730127, ..., 0.27117756, 0.51428294,
        0.46941733], dtype=float32),
 array([0.72856605, 0.25492337, 0.9054919 , ..., 0.33932757, 0.47629607,
        0.41199732], dtype=float32),
 array([0.73929745, 0.2237292 , 0.9052295 , ..., 0.28358278, 0.5110287 ,
        0.37440345], dtype=float32),
 array([0.7509796 , 0.28724867, 0.8912888 , ..., 0.30526415, 0.4860767 ,
        0.38827506], dtype=float32)]

In [48]:
del dtest, cv_generator, test, train

In [55]:
# Predictions stored against each cross validation iteration finally gets aeveraged
# and target column is set with that averaged predictions
submission["target"] = np.mean(np.column_stack(test_predictions), axis=1)

# Checks for sumbission file before saving
submission

,id,target
0,1000000,0.740403
1,1000001,0.256948
2,1000002,0.899050
3,1000003,0.850196
4,1000004,0.262870
...,...,...
499995,1499995,0.971875
499996,1499996,0.816845
499997,1499997,0.296109
499998,1499998,0.511304


In [56]:
# Saves test predictions
submission.to_csv("./submission.csv", index=False)